In [1]:
import pandas as pd
import numpy as np
import math
import shortest_path as sp
import sc_pca

In [2]:

sc_pca.sc_pca("/home/ozgun/Downloads/test-data.tsv", "/home/ozgun/Downloads/test-data-pca.tsv", 3)

sp.write_distance_matrix("/home/ozgun/Downloads/test-data-pca.tsv", "/home/ozgun/Downloads/test-data-dist.tsv")

traj, time = sp.get_traj_and_time("/home/ozgun/Downloads/test-data-dist.tsv", "C1_1", "C2_1")

In [3]:

sp.write_changes("/home/ozgun/Downloads/test-data.tsv", traj, time, "/home/ozgun/Downloads/test-data-diffexp.tsv")
